In [ ]:
import os

In [ ]:
patent_abstract = """
Biometric Vehicle Access System
Abstract:
The Biometric Vehicle Access System (BVAS) is an innovative technology designed to revolutionize traditional vehicle security and access methods.
This system employs advanced biometric authentication, including fingerprint and facial recognition, to ensure secure and convenient entry and ignition processes.
BVAS enhances vehicle security by replacing traditional key-based and electronic fob systems with a seamless and personalized biometric verification process. The technology integrates biometric sensors into door handles, steering wheels, and ignition systems, allowing for quick and reliable user authentication. The BVAS not only provides an additional layer of security against unauthorized access but also enhances user convenience by eliminating the need for physical keys or key fobs. Users can effortlessly unlock, start, and operate their vehicles through a simple and rapid biometric scan. The system is designed with robust anti-spoofing measures to prevent unauthorized access attempts.
Furthermore, BVAS contributes to the growing trend of biometric integration in smart vehicles, aligning with the industry's commitment to innovation, user experience, and safety. As vehicles continue to evolve into interconnected and autonomous entities, BVAS sets a new standard for personalized and secure access, catering to the increasing demand for sophisticated yet user-friendly solutions in the automotive sector.
"""


In [ ]:
from openai import OpenAI
oai_client = OpenAI()

oai_client.embeddings.create(
        model="text-embedding-ada-002",
        input=patent_abstract
    )


In [ ]:
import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

embedding_function = OpenAIEmbeddingFunction(api_key=os.environ.get('OPENAI_API_KEY'),
                                             model_name="text-embedding-ada-002")


chroma_client = chromadb.Client()
vector_store = chroma_client.get_or_create_collection(name="Patents",
                                                      embedding_function=embedding_function)

In [ ]:
vector_store.add("patent_info", documents=patent_abstract)

In [ ]:
from trulens_eval import Tru
from trulens_eval.tru_custom_app import instrument
tru = Tru()
#tru.reset_database() #Reset the database. Clears all tables.


In [ ]:
class RAG_from_scratch:
    @instrument
    def retrieve(self, query: str) -> list:
        """
        Retrieve relevant text from vector store.
        """
        results = vector_store.query(
        query_texts=query,
        n_results=2
    )
        return results['documents'][0]

    @instrument
    def generate_completion(self, query: str, context_str: list) -> str:
        """
        Generate answer from context.
        """
        completion = oai_client.chat.completions.create(
        model="gpt-4",
        temperature=0,
        messages=
        [
            {"role": "user",
            "content": 
            f"The following abstract descripes a concept for a novel invention: \n"
            f"---------------------\n"
            f"{context_str}"
            f"\n---------------------\n"
            f"Given this information, please execute the following instructions: {query}"
            }
        ]
        ).choices[0].message.content
        return completion

    @instrument
    def query(self, query: str) -> str:
        context_str = self.retrieve(query)
        completion = self.generate_completion(query, context_str)
        return completion

rag = RAG_from_scratch()

In [ ]:
from trulens_eval import Feedback, Select
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.openai import OpenAI as fOpenAI

import numpy as np

# Initialize provider class
fopenai = fOpenAI()

grounded = Groundedness(groundedness_provider=fopenai)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = (
    Feedback(fopenai.relevance_with_cot_reasons, name = "Answer Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on_output()
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(fopenai.qs_relevance_with_cot_reasons, name = "Context Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)

In [ ]:
from trulens_eval import TruCustomApp
tru_rag = TruCustomApp(rag,
    app_id = 'RAG Key Words',
    feedbacks = [f_groundedness, f_qa_relevance, f_context_relevance])

In [ ]:
with tru_rag as recording:
    rag.query("Name 5 key words based on this abstract, that I can use for the search in a patent database. Optimize the key words to get back more results. Result as python string.")

In [ ]:
tru.get_leaderboard(app_ids=["RAG Key Words"])

In [ ]:

tru_rag = TruCustomApp(rag,
    app_id = 'RAG Classifications',
    feedbacks = [f_groundedness, f_qa_relevance, f_context_relevance])

In [ ]:
with tru_rag as recording:
    rag.query("Name 5 CPC classifications based on this abstract, that I can use for the search in a patent database. \
Please give me a python string for the codes of the 5 most relevant \
CPC classifications to a possible patent.")

In [ ]:
tru.get_leaderboard(app_ids=["RAG Classifications"])

In [ ]:

tru_rag = TruCustomApp(rag,
    app_id = 'RAG Patent Comparison',
    feedbacks = [f_groundedness, f_qa_relevance, f_context_relevance])

In [ ]:
with tru_rag as recording:
    rag.query(f"""
'The following texts are abstracts from patent specifications. Your task is to compare the novel invention from above to all the other abstracts from below. \n
It is important that you focus on comparing the concepts that the abstracts describe, not the way they are written. \n
Rank the remaining abstracts on how well they match with the Testing Abstract by giving them a rating from 0 to 10 points. \n        
0 meaning they have absolutely nothing in common and 10 meaning they basically describe the exact same idea.\n  
Your output should be a python dictionary with the title "comparison", each element hast the Abstract number as key and the rating as value.\n        
I want to convert your output string to an actual dictionary, so make sure the formatting is right.\n\n        
        
US20180201225A1: "A method for authorizing a driver to activate at least one system of a vehicle, includes a preliminary phase that includes an initialization step, an enrollment step scanning enrollment biometric data of the driver, an access step for the driver to enter the vehicle, and a storage step registering enrollment biometric data in memory. The method also includes a subsequent phase that includes an authentication step scanning authentication biometric data of the driver and comparing said authentication biometric data with the enrollment biometric data in the memory, then an activation step wherein, if the authentication biometric data match with the enrollment biometric data, in authorizing the driver to activate the at least one system of the vehicle, or a refusal step wherein, if the authentication biometric data does not match with the enrollment biometric data, in forbidding the driver to activate the at least one system of the vehicle."\nUS8952781B2: "A biometrically authenticated access control in which a wireless authentication signal is provided from a primary instrumentality of access, only after a dual-stage biometric verification of the user\'s identity is performed. In one embodiment, an accessing device includes memory for storing a device identification code and an authentication code, along with first and second biometric templates corresponding to biometric samples from a user. In another embodiment, an accessing device includes memory for storing a device identification code and more than one authentication code, for separate users, along with first and second biometric templates corresponding to biometric samples from multiple users. In order to gain access to a secured resource, a user undergoes first and second biometric sampling to generate biometric data to be compared with the first and second biometric templates."\nCN111231893B: "The invention relates to a method (17) for operating a shared vehicle (1) and to a corresponding shared vehicle (1). The invention provides that the biometric data of the holder of the driver\'s license (14) is read from the driver\'s license (14) carried by the user (5) by means of a data acquisition device (13). Furthermore, at least one biometric of the user (5) is detected by means of the identification device (13) during the stay of the user in the area of the shared vehicle (1). Next, to verify the read data, the data is compared with the detected biometric features. In the event of agreement between the read data and the detected biometric feature, i.e. when a positive test result is obtained, the shared vehicle (1) is activated for the user (5) for use."\nUS8694793B2: "Aspects and embodiments of the present disclosure provide devices and methods for biometric authentication of a user during access control transactions. In one aspect, an access control processor device, comprising a biometric input sensor configured to receive user biometric information; a biometric verification processor configured to authenticate the input user biometric information; and a communication element configured to activate when the biometric information entered into the biometric verification system is authenticated and maintain an inactive status for the communication element on the payment processor device when the biometric information entered into the biometric verification system is not authenticated."\nEP1705085B1: "A driver authenticating apparatus has a camera for capturing a facial image of a person in a driver seat, and a controller connected to the camera. The controller permits a driver to start an vehicle engine if the captured facial image matches a pre-registered facial image of an authorized driver. When restarting an engine with the driver present in the driver seat, the apparatus allows the driver to start the vehicle engine without an authenticating procedure."\nUS9654468B2: "Systems and methods for secure remote biometric authentication are provided. A network-based biometric authentication platform stores biometric templates for individuals which have been securely enrolled with the authentication platform. A plurality of sensor platforms separately establishes secure communications with the biometric authentication platform. The sensor platform can perform a biometric scan of an individual and generate a biometric authentication template. The sensor platform then requests biometric authentication of the individual by the biometric authentication platform via the established secure communications. The biometric authentication platform compares the generated biometric template to one or more of the enrolled biometric templates stored in memory at the biometric authentication platform. The result of the authentication is then communicated to the requesting sensor platform via the established secure communications."\nUS11397800B2: "A removable card-enabled BPID Security Device integrates a removable card reader with a biometric authentication component to provide secured access to electronic systems. The device allows for an individual to insert a removable card into an aperture in the physical enclosure of the BPID Security Device, allowing the removable card and the BPID Security Device to electronically communicate with each other. The BPID Security Device is based on a custom application specific integrated circuit that incorporates removable card terminals, such that the BPID Security Device can communicate directly with an inserted removable card. In an alternative embodiment of the invention, the BPID Security Device is based on a commercial off-the-shelf microprocessor, and may communicate with a commercial off-the-shelf microprocessor removable card receiver using a serial, USB, or other type of communication protocol. The device allows for enrolling a user\'s credentials onto the BPID Security Device and for authenticating an individual using the BPID Security Device."\nUS20160311400A1: "The invention relates to a method for authenticating a driver (2) in a motor vehicle (1), having a detection device (10) which is arranged in the motor vehicle (1) and has the purpose of detecting actual data (50) of the driver (2) which is transmitted during the authentication to a checking device (20) which is arranged in an external station (3) outside the motor vehicle (1), wherein the checking device (20) compares the actual data (50) with setpoint data (60), and when the actual data (50) corresponds to the setpoint data (60) an enable signal (70) is transmitted from the external station (3) to the motor vehicle (1), as a result of which a starting process of the motor vehicle (1) for the driver (2) is made possible."\nUS9376090B2: "The invention relates to a method for authenticating a driver (2) in a motor vehicle (1) by means of a recognition device (10) disposed in the motor vehicle (1) for collecting actual data (50) of the driver (2) which are transmitted during the authentication to a checking device (20) disposed in an external station (3) outside the motor vehicle (1), wherein the checking device (20) compares the actual data (50) with the target data (60) and in the event of conformity of the actual data (50) with the target data (60) an enabling signal (70) is sent from the external station (3) to the motor vehicle (1), thus enabling the driver (2) to start the motor vehicle (1)."\n'
""")

In [ ]:
tru.get_leaderboard(app_ids=["RAG Patent Comparison"])